In [1]:
import boto3
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import os

import sagemaker
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
#bucket = '20210520-sagemaker-yolo-ddp'
prefix = 'data'

role = sagemaker.get_execution_role()

session = boto3.session.Session()
region = session.region_name
print(f"AWS region:{region}")

AWS region:us-east-1


In [12]:
#image = "yolov5-sagemaker-109-cu100-4"  # Example: mask-rcnn-smdataparallel-sagemaker
#image = "yolov5-sagemaker-109-cu111-4"
image = "yolov5-sagemaker-109-cu100-env"
tag = "pt1.8"  # Example: pt1.8

In [13]:
!docker system prune -a -f --volumes
!docker volume rm "docker volume ls -q -f dangling=true"

Deleted Images:
untagged: 815969174475.dkr.ecr.us-east-1.amazonaws.com/yolov5-sagemaker-109-cu100:pt1.8
untagged: 815969174475.dkr.ecr.us-east-1.amazonaws.com/yolov5-sagemaker-109-cu100@sha256:7bd1f287c5928a77a6eeb726ba5684baf24da7e32a084a315ebcb385955ad114
untagged: yolov5-sagemaker-109-cu100:latest
deleted: sha256:1a3fa9f1c9da70be1afa395e10d4daef0d6e4ae4e3b2e425cf4a8080f8f45873
deleted: sha256:9b96c8929e242e7b0f526cad97e23c5ae5b549263c2766519cb5a2726953393b
deleted: sha256:c2c27063d8c01e3c6d1ee85bdb19da161b5fae201a60bbec94a093bad3aa9e01
deleted: sha256:c31fca03e82940ed9a8a7df2b28b0c92b55dd094e97607f1c08c9fba44a14ded
deleted: sha256:11b829437e820ef6d1beda926096811a1fea0ba1cd7078dd8823a2243178dc8d
deleted: sha256:3f972b8071bc3a06d18f071ce5f43487e5bb5f25674d7ae22ac976e0a1aabea1
deleted: sha256:77dc9eef25e1459220c722ed7c4fc100f88578a911f597434abda4b981a2915f
deleted: sha256:63467a17ca784a3618a6224c8404eea5821e6ac3b20574b3fc76e891ce98bdeb
untagged: 763104351884.dkr.ecr.us-east-1.amazonaws

In [14]:
!pygmentize ./Dockerfile

ARG region

# FROM 763104351884.dkr.ecr.${region}.amazonaws.com/pytorch-training:1.8.1-gpu-py36-cu111-ubuntu18.04
FROM 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.7.1-gpu-py36-cu110-ubuntu18.04

#RUN cd /root && \
#	git clone --recursive https://github.com/tkazusa/yolov5-sagemaker-distributed-data-parallel.git && \
#    chmod -R 775 yolov5-sagemaker-distributed-data-parallel && \
#	cd yolov5-sagemaker-distributed-data-parallel && \
#	pip install -r requirements.txt &&\
#    pip install -U smdebug
    

# Install python dependenies
COPY requirements.txt .
RUN python -m pip install --upgrade pip
RUN pip install --no-cache -r requirements.txt coremltools onnx gsutil notebook
RUN pip install -U smdebug

ENV SAGEMAKER_INSTANCE_TYPE ml.p3.16xlarge


In [15]:
!docker images

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE


In [ ]:
%%time
! chmod +x build_and_push.sh; bash build_and_push.sh {region} {image} {tag}

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  191.3MB
Step 1/7 : ARG region
Step 2/7 : FROM 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.7.1-gpu-py36-cu110-ubuntu18.04
1.7.1-gpu-py36-cu110-ubuntu18.04: Pulling from pytorch-training

57c49d0f: Pulling fs layer 
40447d26: Pulling fs layer 
2f862619: Pulling fs layer 
278deddf: Pulling fs layer 
80049843: Pulling fs layer 
556b2329: Pulling fs layer 
a0c97a55: Pulling fs layer 
78bd0b24: Pulling fs layer 
6c31766d: Pulling fs layer 
d9c8e84f: Pulling fs layer 
7fdf2ea6: Pulling fs layer 
c41da723: Pulling fs layer 
0befbc6f: Pulling fs layer 
cf9fbe3c: Pulling fs layer 
c7df16f1: Pulling fs layer 
d58b12f9: Pulling f

In [ ]:
import os
from sagemaker.pytorch import PyTorch
from sagemaker.local import LocalSession

client = boto3.client("sts")
account = client.get_caller_identity()["Account"]

instance_type = "ml.p3.16xlarge"  # Other supported instance type: ml.p3.16xlarge, ml.p4d.24xlarge
instance_count = 1  # You can use 2, 4, 8 etc.
docker_image = f"{account}.dkr.ecr.{region}.amazonaws.com/{image}:{tag}"  # YOUR_ECR_IMAGE_BUILT_WITH_ABOVE_DOCKER_FILE
job_name = "pytorch-sm-yolo-ddp14"

# SageMakerの操作で使用するローカル用セッション
local_session = LocalSession()

In [ ]:
docker_image

In [ ]:
estimator = PyTorch(
    entry_point="train.py",
    role=role,
    volume_size=700,
    source_dir=".",
    image_uri=docker_image,
    instance_count=instance_count,
    instance_type=instance_type,
    framework_version="1.8.1",
    sagemaker_session=sagemaker_session,
    distribution={"smdistributed": {"dataparallel": {"enabled": True}}},
    debugger_hook_config=False
)

In [ ]:
data_uri = "s3://sagemaker-us-east-1-815969174475/coco/"
#data_uri = "s3://sagemaker-us-east-1-815969174475/sagemaker-handson-kazusa/"

data_channels = {"data": data_uri}

estimator.fit(inputs=data_channels, job_name=job_name, wait=False)